In [3]:
import pandas as pd
import numpy as np
import os

DIST = 275

pd.set_option('display.max_columns', None)

In [3]:
def load_dataset(low_memory=True):
    file_name = fr'C:\machine learning\gb_greyhound\grayhound\data\train\datasets\dataset_{DIST}.0.csv'
    return pd.read_csv(file_name, low_memory=low_memory)

main_df = load_dataset(low_memory=False)

In [30]:
columns_to_drop = ['SP', 'dogBorn', 'dogColour', 'dogId', 'dogSeason', 
           'dogSex', 'dogSire', 'dogSire', 'meetingId', 
           'ownerName', 'raceForecast', 'raceId', 'raceNumber', 
           'raceHandicap', 'racePrizes', 'raceTricast', 'raceType', 
           'resultAdjustedTime', 'resultMarketCnt', 'resultMarketPos',
           'trainerName', 'trapHandicap', 'raceTime', 'raceTime',
            'resultSectionalTime', 'raceDistance', 'resultComment',
            'trackName'
]

df = main_df.drop(columns_to_drop, axis=1)
df.head()

,dogName,raceClass,raceDate,raceGoing,resultBtnDistance,resultDogWeight,resultPosition,resultPriceDenominator,resultPriceNumerator,resultRunTime,trapNumber
0,NaN,D2,10/01/2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
1,Leemount Shaw,D2,10/01/2014,NaN,NaN,33.1,1.0,2.0,9.0,16.17,1
2,Marlow Light,D2,10/01/2014,NaN,1 3/4,31.7,2.0,1.0,5.0,16.31,3
3,Midnight Boss,D2,10/01/2014,NaN,1 1/2,30.0,3.0,11.0,10.0,16.43,5
4,Safe Impact,D2,10/01/2014,NaN,1 3/4,35.7,4.0,1.0,3.0,16.57,4


In [31]:
columns = df.columns

for col in columns:
    print(col, ':', df[col].nunique())

dogName : 4159
raceClass : 11
raceDate : 2418
raceGoing : 26
resultBtnDistance : 84
resultDogWeight : 215
resultPosition : 7
resultPriceDenominator : 13
resultPriceNumerator : 24
resultRunTime : 374
trapNumber : 6


In [32]:
df.replace(['', ' ', np.nan], np.nan)
df_cleaned = df.dropna(subset=['dogName', 'raceClass', 'resultPosition', 'resultPriceDenominator', 'resultPriceNumerator', 'raceDate'])

In [33]:
df_cleaned['resultPosition'].value_counts()

resultPosition
1.0    7945
2.0    7908
3.0    7901
4.0    7874
5.0    7758
6.0    6458
0.0      74
Name: count, dtype: int64

In [34]:
df_cleaned = df_cleaned[df_cleaned['resultPosition'] != 0.0]

In [35]:
df_cleaned['raceClass'].unique()

array(['D2', 'D3', 'D4', 'OR', 'IT', 'D1', 'D5', 'B3', 'OR2', 'OR3', 'IV'],
      dtype=object)

In [36]:
len(df_cleaned[df_cleaned['raceClass'] == 'B3'])

6

In [37]:
df_cleaned = df_cleaned[df_cleaned['raceClass'] != 'B3']

In [38]:
df_cleaned.loc[:, 'raceDate'] = pd.to_datetime(df_cleaned['raceDate'], format='%d/%m/%Y')
df_cleaned.loc[:, 'forecast'] = df_cleaned['resultPriceNumerator'] / df_cleaned['resultPriceDenominator']

In [39]:
def process_dog_data(df):
    dog_names = df['dogName'].unique()

    results = []
    
    for dog in dog_names:
        dog_data = df[df['dogName'] == dog].sort_values(by='raceDate')
        
        dog_results = []
        for i, row in dog_data.iterrows():
            current_race_date = row['raceDate']
            
            previous_races = dog_data[dog_data['raceDate'] < current_race_date].tail(5)
            
            by = previous_races['resultBtnDistance'].tolist()
            finished = previous_races['resultPosition'].tolist()
            going = previous_races['raceGoing'].tolist()
            price_dens = previous_races['resultPriceDenominator'].tolist()
            price_nums = previous_races['resultPriceNumerator'].tolist()
            race_grade = previous_races['raceClass'].tolist()
            run_time = previous_races['resultRunTime'].tolist()
            trap = previous_races['trapNumber'].tolist()
            weight = previous_races['resultDogWeight'].tolist()

            while len(by) < 5:
                by.append(np.nan)
                finished.append(np.nan)
                going.append(np.nan)
                price_dens.append(np.nan)
                price_nums.append(np.nan)
                race_grade.append(np.nan)
                run_time.append(np.nan)
                trap.append(np.nan)
                weight.append(np.nan)

            result = (by + finished + going + 
                      price_dens + price_nums +
                      race_grade + run_time + 
                      trap + weight)

            dog_results.append(result)
        
        dog_data_results = pd.DataFrame(dog_results, columns=[
            'by_1', 'by_2', 'by_3', 'by_4', 'by_5',
            'finished_1', 'finished_2', 'finished_3', 'finished_4', 'finished_5',
            'going_1', 'going_2', 'going_3', 'going_4', 'going_5',
            'price_dens_1', 'price_dens_2', 'price_dens_3', 'price_dens_4', 'price_dens_5',
            'price_nums_1', 'price_nums_2', 'price_nums_3', 'price_nums_4', 'price_nums_5',
            'race_grade_1', 'race_grade_2', 'race_grade_3', 'race_grade_4', 'race_grade_5',
            'run_time_1', 'run_time_2', 'run_time_3', 'run_time_4', 'run_time_5',
            'trap_1', 'trap_2', 'trap_3', 'trap_4', 'trap_5',
            'weight_1', 'weight_2', 'weight_3', 'weight_4', 'weight_5'
        ])
        dog_data.reset_index(drop=True, inplace=True)
        dog_data_results.reset_index(drop=True, inplace=True)
        
        combined_data = pd.concat([dog_data, dog_data_results], axis=1)
        results.append(combined_data)
    
    final_df = pd.concat(results).sort_index()

    return final_df

df_cleaned = df_cleaned.replace(['', ' ', None], np.nan)
df_full = process_dog_data(df_cleaned)

C:\Users\milie\AppData\Local\Temp\ipykernel_21036\1863837943.py:64: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned = df_cleaned.replace(['', ' ', None], np.nan)


In [40]:
df_full = df_full.dropna(subset=['finished_1'])

In [41]:
df_full = df_full.replace([None], np.nan)

In [42]:
def fill_miss_values(row):
    last_results = None
    size = None
    start_index = None

    for i in range(2, 6):
        if pd.isna(row[f'finished_{i}']):
            last_results = row[[f'by_{i - 1}', f'finished_{i - 1}', 
                                f'going_{i - 1}', f'price_dens_{i - 1}', f'price_nums_{i - 1}',
                                f'race_grade_{i - 1}', f'run_time_{i - 1}', f'trap_{i - 1}', f'weight_{i - 1}']]
            size = 5 - i + 1
            start_index = i
            break
    
    if size is not None:
        for j in range(size):
            idx = start_index + j
            row[f'by_{idx}'] = last_results[f'by_{start_index - 1}']
            row[f'finished_{idx}'] = last_results[f'finished_{start_index - 1}']
            row[f'going_{idx}'] = last_results[f'going_{start_index - 1}']
            row[f'price_dens_{idx}'] = last_results[f'price_dens_{start_index - 1}']
            row[f'price_nums_{idx}'] = last_results[f'price_nums_{start_index - 1}']
            row[f'race_grade_{idx}'] = last_results[f'race_grade_{start_index - 1}']
            row[f'run_time_{idx}'] = last_results[f'run_time_{start_index - 1}']
            row[f'trap_{idx}'] = last_results[f'trap_{start_index - 1}']
            row[f'weight_{idx}'] = last_results[f'weight_{start_index - 1}']
    
    return row

In [43]:
df_full = df_full.apply(fill_miss_values, axis=1)

In [44]:
for i in range(5):
    df_full[f'odds_{i+1}'] = df_full[f'price_nums_{i+1}'] / df_full[f'price_dens_{i+1}']

columns_to_drop = ['dogName', 'raceDate', 'resultBtnDistance', 
                   'raceGoing', 'resultPriceDenominator', 
                   'resultPriceNumerator', 'resultRunTime',
                   'resultDogWeight', 'price_dens_1', 'price_dens_2',
                   'price_dens_3', 'price_dens_4', 'price_dens_5',
                   'price_nums_1', 'price_nums_2', 'price_nums_3',
                   'price_nums_4', 'price_nums_5']
df_full = df_full.drop(columns_to_drop, axis=1)

In [45]:
import re

def convert_dist_by(sp):
    try:
        if isinstance(sp, float) and pd.isna(sp):
            return np.nan
        elif isinstance(sp, (int, float)):
            return sp
        elif isinstance(sp, str):
            parts = sp.split()
            
            if len(parts) == 2:  # Формат "3 3/4"
                whole_part = int(parts[0])
                fraction_part = parts[1]
                
                if '/' in fraction_part:
                    num, den = fraction_part.split('/')
                    fraction_value = int(num) / int(den)
                    return whole_part + fraction_value
                else:
                    return np.nan
            
            elif len(parts) == 1:  # Формат "3/4" или "3"
                if '/' in parts[0]:  # Если это дробь
                    num, den = parts[0].split('/')
                    return int(num) / int(den)
                else:  # Если это просто целое число
                    return float(parts[0])
            else:
                return np.nan
        else:
            return np.nan
    except (ValueError, IndexError) as e:
        return np.nan
    except Exception as e:
        return np.nan

In [46]:
for i in range(5):
    df_full.loc[:, f'by_{i+1}'] = df_full[f'by_{i+1}'].apply(convert_dist_by).round(2)

In [47]:
def set_adv_lagg(pos, by):
    if by is None or pd.isna(by):
        return np.nan
    result = np.round(by * 0.8, 2) if pos == 1 else np.round(by * -0.8, 2)
    return result

In [48]:
for i in range(5):
    df_full[f'by_{i+1}'] = df_full.apply(lambda row: set_adv_lagg(row[f'finished_{i+1}'], row[f'by_{i+1}']), axis=1)

In [49]:
import joblib

def save_encoder(encoder) -> None:
    dir_name = os.path.join("grayhound", "encoders")
    os.makedirs(dir_name, exist_ok=True)
    joblib.dump(encoder, os.path.join(dir_name, f"encoder_{DIST}.pkl"))

In [50]:
from sklearn.preprocessing import OrdinalEncoder

columns_cat = ['raceClass', 'race_grade_1', 'race_grade_2', 'race_grade_3', 'race_grade_4', 'race_grade_5']
df_cat = df_full[columns_cat]

encoder = OrdinalEncoder()
encoder.fit(df_cat)

save_encoder(encoder)

df_encoded = encoder.transform(df_cat)
df_encoded_df = pd.DataFrame(df_encoded, columns=columns_cat)
df_full[columns_cat] = df_encoded_df

In [51]:
y = df_full['resultPosition']
df_full = df_full.drop(['resultPosition'], axis=1)

In [52]:
def save_imputer(imputer):
    dir_name = os.path.join("grayhound", "imputers")
    os.makedirs(dir_name, exist_ok=True)
    joblib.dump(imputer, os.path.join(dir_name, f"imputer_{DIST}.pkl"))

In [53]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3, weights='distance', keep_empty_features=False)
imputer.fit(df_full)

save_imputer(imputer)

X = imputer.transform(df_full)

In [27]:
df_ready = pd.DataFrame(X, columns=df_full.columns)

In [28]:
def save_dataset(dataset):
    dir_name = os.path.join("grayhound", "datasets_to_use")
    os.makedirs(dir_name, exist_ok=True)
    file_path = os.path.join(dir_name, f"dataset_{DIST}.csv")
    dataset.to_csv(file_path)

In [4]:
def load_dataset():
    return pd.read_csv(os.path.join("grayhound", "datasets_to_use",  f"dataset_{DIST}.csv"))

In [29]:
save_dataset(df_ready)

In [5]:
df_ready = load_dataset()

In [13]:
df_ready.head()

,Unnamed: 0,raceClass,resultPosition,trapNumber,forecast,by_1,by_2,by_3,by_4,by_5,finished_1,finished_2,finished_3,finished_4,finished_5,going_1,going_2,going_3,going_4,going_5,race_grade_1,race_grade_2,race_grade_3,race_grade_4,race_grade_5,run_time_1,run_time_2,run_time_3,run_time_4,run_time_5,trap_1,trap_2,trap_3,trap_4,trap_5,weight_1,weight_2,weight_3,weight_4,weight_5,odds_1,odds_2,odds_3,odds_4,odds_5
0,0,3.0,6.0,3.0,3.0,-0.8,-0.8,-0.8,-0.8,-0.8,3.0,3.0,3.0,3.0,3.0,-5.0,-5.0,-5.0,-5.0,-5.0,3.0,3.0,3.0,3.0,3.0,17.16,17.16,17.16,17.16,17.16,2.0,2.0,2.0,2.0,2.0,32.2,32.2,32.2,32.2,32.2,4.0,4.0,4.0,4.0,4.0
1,1,3.0,4.0,3.0,8.0,-0.6,-0.6,-0.6,-0.6,-0.6,6.0,6.0,6.0,6.0,6.0,-5.0,-5.0,-5.0,-5.0,-5.0,3.0,3.0,3.0,3.0,3.0,17.34,17.34,17.34,17.34,17.34,4.0,4.0,4.0,4.0,4.0,25.5,25.5,25.5,25.5,25.5,8.0,8.0,8.0,8.0,8.0
2,2,3.0,6.0,2.0,3.0,-0.4,-0.4,-0.4,-0.4,-0.4,2.0,2.0,2.0,2.0,2.0,-15.0,-15.0,-15.0,-15.0,-15.0,3.0,3.0,3.0,3.0,3.0,17.97,17.97,17.97,17.97,17.97,2.0,2.0,2.0,2.0,2.0,28.2,28.2,28.2,28.2,28.2,2.5,2.5,2.5,2.5,2.5
3,3,3.0,6.0,2.0,4.5,-2.6,-2.6,-2.6,-2.6,-2.6,4.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,3.0,16.76,16.76,16.76,16.76,16.76,1.0,1.0,1.0,1.0,1.0,29.2,29.2,29.2,29.2,29.2,7.0,7.0,7.0,7.0,7.0
4,4,3.0,3.0,4.0,4.0,-3.0,-3.0,-3.0,-3.0,-3.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,3.0,16.55,16.55,16.55,16.55,16.55,4.0,4.0,4.0,4.0,4.0,35.4,35.4,35.4,35.4,35.4,6.0,6.0,6.0,6.0,6.0


In [6]:
from sklearn.model_selection import train_test_split


y = df_ready['resultPosition']
X = df_ready.drop(['Unnamed: 0', 'resultPosition'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [7]:
y.dtype

dtype('float64')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'n_estimators': [3, 10, 30, 50, 100, 120],
        'max_features': [5, 10, 15, 25, 44],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10, 15, 20],
    },
    {
        'bootstrap': [False],
        'n_estimators': [3, 10, 30, 50, 100, 120],
        'max_features': [5, 10, 15, 25, 44],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10, 15, 20],
    }
]

forest_reg = RandomForestClassifier(random_state=42)
grd_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_log_loss')
grd_search.fit(X, y)

In [ ]:
final_model = grd_search.best_estimator_

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(final_model,
                         X_test, y_test,
                         scoring="neg_log_loss", cv=10)
forest_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores' sum:", scores),
    print("Mean:", scores.mean()),
    print("Std:", scores.std())

display_scores(forest_scores)

In [ ]:
def save_model(model):
    file_name = f"model_{DIST}.pkl"
    file_path = os.path.join("grayhound", "models", "random_forest_class")
    os.makedirs(file_path, exist_ok=True)
    joblib.dump(model, os.path.join(file_path, file_name))

In [ ]:
grd_search.best_params_

In [ ]:
save_model(final_model)